In [3]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os 
from PIL import Image
%matplotlib inline

import base64
import datetime
import time
from azure.storage import CloudStorageAccount, AccessPolicy
from azure.storage.blob import BlockBlobService, PageBlobService, AppendBlobService
from azure.storage.models import CorsRule, Logging, Metrics, RetentionPolicy, ResourceTypes, AccountPermissions
from azure.storage.blob.models import BlobBlock, ContainerPermissions, ContentSettings

In [4]:
import os

import json
with open('config.json', 'r') as f:
    config = json.load(f)

root_dir = config['FOLDERS']['ROOT_DIR']
photos_dir = config['FOLDERS']['PATH']
version = config['VERSION']
filters_dir = config['FOLDERS']['FILTERS']
filters_dir = r'C:\Users\thema\Documents\Ralph Lauren\Filters'
photos_dir = r'C:\Users\thema\Documents\Ralph Lauren\Original'
save_dir = r'C:\Users\thema\Documents\Ralph Lauren\Filtered'

In [5]:
def save(img,path,name):
    n = r'\{}'.format(name)
    cv2.imwrite(path+n,img)

def load(path,name):
    n = r'\{}'.format(name)
    return cv2.imread(path+n)

def deleteFile(path,name):
    n = r'\{}'.format(name)
    if os.path.exists(path+n):
        print('Deleting file ' +path+n)
        os.remove(path+n)

def saveToBlob(block_blob_service, local_path,local_file_name,container_name):
    # Create a file in Documents to test the upload and download.
    full_path_to_file =os.path.join(local_path, local_file_name)
    print("Temp file = " + full_path_to_file)
    print("\nUploading to Blob storage as blob: " + local_file_name)
    # Upload the created file, use local_file_name for the blob name
    block_blob_service.create_blob_from_path(container_name, local_file_name, full_path_to_file, content_settings=ContentSettings(content_type='image/png'))

def AddFilter(image,background):
    img1 = image
    if img is not None:
    img2 = background
    
    # I want to put logo on top-left corner, So I create a ROI

    rows,cols,channels = img2.shape
    x,y,c = img1.shape

    #ratio = x/cols
    #dst = cv2.resize(img2, (np.int(ratio*cols),np.int(ratio*rows)), interpolation = cv2.INTER_CUBIC)
    #rows,cols,channels = dst.shape
    roi = img1[(y-rows):y,np.int((x/2)-(cols/2)):np.int((x/2)-(cols/2))+cols]
    # Now create a mask of logo and create its inverse mask also
    img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(img2gray, 180, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)

    # Now black-out the area of logo in ROI
    img1_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)

    # Take only region of logo from logo image.
    img2_fg = cv2.bitwise_and(img2,img2,mask = mask)

    # Put logo in ROI and modify the main image
    dst = cv2.add(img1_bg,img2_fg)
    img1[(y-rows):y,np.int((x/2)-(cols/2)):np.int((x/2)-(cols/2))+cols] = dst



In [4]:
#Blob container config
account_key = config['AZURE']['BLOBSTORAGE']['ACCOUNT_KEY']
account_name = config['AZURE']['BLOBSTORAGE']['ACCOUNT_NAME']
container_name = config['AZURE']['BLOBSTORAGE']['ACCOUNT_NAME']
block_blob_service = BlockBlobService(account_name= account_name, account_key=account_key) 

#loading filters
foreground = Image.open(filters_dir+r'/WPFilter.png')
dim = [foreground.height,foreground.width]

#download all blobs and add filter
generator = block_blob_service.list_blobs(container_name)

for blob in generator:
    print("\t Blob name: " + blob.name)
    n = r'\{}'.format( blob.name)
    full_path_to_file2 = os.path.join(photos_dir+ n)
    print("\nDownloading blob to " + full_path_to_file2)
    #download one picture
    block_blob_service.get_blob_to_path(container_name, blob.name, full_path_to_file2)
    #Apply filters
    filename = r""+blob.name
    print("Applying filter to file = " + filename)
    print(filename)
    #load picture to modify
    background = Image.open(photos_dir+ r'\{}'.format(filename))
    print(photos_dir)
    if background is not None:
        background_r = background.resize(dim)
        background_r.paste(foreground, (0, 0),foreground )
        local_file_name = "Filter"+blob.name
        local_file_name = local_file_name.split('.')[0]
        local_file_name = local_file_name +".png"
        full_path_to_file = os.path.join(save_dir+r'\{}'.format(local_file_name))
        background_r.save(full_path_to_file,"PNG")
        #upload to blob storage
       
        print("Temp file = " + full_path_to_file)
        print("\nUploading to Blob storage file:  " + local_file_name)
        ##save to blob modified picture
        saveToBlob(block_blob_service,save_dir,local_file_name,container_name)
        #deleting local files
        deleteFile(save_dir,local_file_name)
        deleteFile(photos_dir,filename)
        
    

	 Blob name: 100021810561904.jpg

Applying filter to file = 100021810561904.jpg
100021810561904.jpg
C:\Users\thema\Documents\Ralph Lauren\Original
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100021810561904.png

Uploading to Blob storage file:  Filter100021810561904.png
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100021810561904.png

Uploading to Blob storage as blob: Filter100021810561904.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100021810561904.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Original\100021810561904.jpg
	 Blob name: 100021816503748.jpg

Applying filter to file = 100021816503748.jpg
100021816503748.jpg
C:\Users\thema\Documents\Ralph Lauren\Original
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100021816503748.png

Uploading to Blob storage file:  Filter100021816503748.png
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100021816503748.png

Uploading to Bl

Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100021888282270.png

Uploading to Blob storage file:  Filter100021888282270.png
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100021888282270.png

Uploading to Blob storage as blob: Filter100021888282270.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100021888282270.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Original\100021888282270.jpg
	 Blob name: 100021888732358.jpg

Applying filter to file = 100021888732358.jpg
100021888732358.jpg
C:\Users\thema\Documents\Ralph Lauren\Original
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100021888732358.png

Uploading to Blob storage file:  Filter100021888732358.png
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100021888732358.png

Uploading to Blob storage as blob: Filter100021888732358.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100021888732358.png
Deleting file 

Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022013654268.png

Uploading to Blob storage file:  Filter100022013654268.png
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022013654268.png

Uploading to Blob storage as blob: Filter100022013654268.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022013654268.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Original\100022013654268.jpg
	 Blob name: 100022025983246.jpg

Applying filter to file = 100022025983246.jpg
100022025983246.jpg
C:\Users\thema\Documents\Ralph Lauren\Original
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022025983246.png

Uploading to Blob storage file:  Filter100022025983246.png
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022025983246.png

Uploading to Blob storage as blob: Filter100022025983246.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022025983246.png
Deleting file 

Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022185591410.png

Uploading to Blob storage file:  Filter100022185591410.png
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022185591410.png

Uploading to Blob storage as blob: Filter100022185591410.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022185591410.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Original\100022185591410.jpg
	 Blob name: 100022194709247.jpg

Applying filter to file = 100022194709247.jpg
100022194709247.jpg
C:\Users\thema\Documents\Ralph Lauren\Original
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022194709247.png

Uploading to Blob storage file:  Filter100022194709247.png
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022194709247.png

Uploading to Blob storage as blob: Filter100022194709247.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022194709247.png
Deleting file 

Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022243513065.png

Uploading to Blob storage file:  Filter100022243513065.png
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022243513065.png

Uploading to Blob storage as blob: Filter100022243513065.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022243513065.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Original\100022243513065.jpg
	 Blob name: 100022246572659.jpg

Applying filter to file = 100022246572659.jpg
100022246572659.jpg
C:\Users\thema\Documents\Ralph Lauren\Original
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022246572659.png

Uploading to Blob storage file:  Filter100022246572659.png
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022246572659.png

Uploading to Blob storage as blob: Filter100022246572659.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100022246572659.png
Deleting file 

...

Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029044567749.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Original\100029044567749.jpg
	 Blob name: 100029045705813.jpg

Applying filter to file = 100029045705813.jpg
100029045705813.jpg
C:\Users\thema\Documents\Ralph Lauren\Original
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029045705813.png

Uploading to Blob storage file:  Filter100029045705813.png
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029045705813.png

Uploading to Blob storage as blob: Filter100029045705813.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029045705813.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Original\100029045705813.jpg
	 Blob name: 100029046035440.jpg

Applying filter to file = 100029046035440.jpg
100029046035440.jpg
C:\Users\thema\Documents\Ralph Lauren\Original
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter1000290460354

Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029062623324.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Original\100029062623324.jpg
	 Blob name: 100029065989125.jpg

Applying filter to file = 100029065989125.jpg
100029065989125.jpg
C:\Users\thema\Documents\Ralph Lauren\Original
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029065989125.png

Uploading to Blob storage file:  Filter100029065989125.png
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029065989125.png

Uploading to Blob storage as blob: Filter100029065989125.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029065989125.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Original\100029065989125.jpg
	 Blob name: 100029067158822.jpg

Applying filter to file = 100029067158822.jpg
100029067158822.jpg
C:\Users\thema\Documents\Ralph Lauren\Original
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter1000290671588

Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029079452241.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Original\100029079452241.jpg
	 Blob name: 100029084553551.jpg

Applying filter to file = 100029084553551.jpg
100029084553551.jpg
C:\Users\thema\Documents\Ralph Lauren\Original
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029084553551.png

Uploading to Blob storage file:  Filter100029084553551.png
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029084553551.png

Uploading to Blob storage as blob: Filter100029084553551.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029084553551.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Original\100029084553551.jpg
	 Blob name: 100029085365200.jpg

Applying filter to file = 100029085365200.jpg
100029085365200.jpg
C:\Users\thema\Documents\Ralph Lauren\Original
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter1000290853652

Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029110502494.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Original\100029110502494.jpg
	 Blob name: 100029110890743.jpg

Applying filter to file = 100029110890743.jpg
100029110890743.jpg
C:\Users\thema\Documents\Ralph Lauren\Original
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029110890743.png

Uploading to Blob storage file:  Filter100029110890743.png
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029110890743.png

Uploading to Blob storage as blob: Filter100029110890743.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029110890743.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Original\100029110890743.jpg
	 Blob name: 100029112810044.jpg

Applying filter to file = 100029112810044.jpg
100029112810044.jpg
C:\Users\thema\Documents\Ralph Lauren\Original
Temp file = C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter1000291128100

Deleting file C:\Users\thema\Documents\Ralph Lauren\Filtered\Filter100029138731560.png
Deleting file C:\Users\thema\Documents\Ralph Lauren\Original\100029138731560.jpg
